# Assignment 7

## Submit as an HTML file

### Print your name below

In [41]:
print("Winnie Lau")

Winnie Lau


### Import the "pandas" "numpy" and "statsmodels.formula.api" libraries

In [42]:
# Write your answer here:

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

#### In the code chunk below read the CSV file named `results.csv` in the `data` <br> folder and print the first 5 rows of the dataset. Browse the dataset.

In [43]:
results = pd.read_csv("data/results.csv")
results.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


### (a)  Check Column Types and Data Cleaning

- Use the function .dtypes to get the column types
- Identify which columns have data types that might need conversion
- The 'milliseconds' column contains string values that should be numeric. Create a new column called 'race_time_ms' that:
    - Converts the column to a numeric data type
    - Replaces any non-numeric values with NaN

In [44]:
# Write your answer here

results.dtypes

results["milliseconds"].str.isnumeric()
subset = results.query("milliseconds.str.isnumeric()==False")
list_unique = pd.unique(subset["milliseconds"])
list_unique


results["race_time_ms"] = results["milliseconds"].replace("\\N", np.nan)
results["race_time_ms"] = pd.to_numeric(results["race_time_ms"])


### (b) Create Categorical Variables

- Create a new column called 'finish_category' that categorizes the race finish positions as follows:
    - Positions 1-3: 'Podium'
    - Positions 4-10: 'Points'
    - Positions 11-20: 'Midfield'
    - Positions >20: 'Backmarker'

Hint: Use the pd.cut() function

In [45]:
# Write your answer here

bins1 = [0,3,10,20,40]
labels1 = ["Podium", "Points", "Midfield", "Backmarker"]

results["finish_category"] = pd.cut(results["positionOrder"],
                                    bins = bins1,
                                    right = True,
                                    labels = labels1)

# ??which column to use?

### (c) Calculate Race Duration
- For rows where 'milliseconds' is available, create a new column <br>
'race_duration_minutes' that converts milliseconds to minutes by dividing <br>
by (1000*60).
- Display the average race duration by 'constructorId' for the top 5 <br>
constructors with the shortest average race times

In [46]:
# Write your answer here

results["race_duration_minutes"] = results["race_time_ms"]/(1000*60)

duration_agg = (results.groupby("constructorId")
                .agg(avg_race_duration = ("race_duration_minutes", "mean"))
                .sort_values(by="avg_race_duration", ascending=True))

duration_agg.head()

,avg_race_duration
constructorId,
35,76.710777
29,77.604125
41,87.046767
16,89.428828
53,89.658852


### (d) Driver Performance Analysis

- Calculate the following statistics for each driver, grouped by 'driverId':
    - Average finishing position
    - Total points
    - Number of races completed
    - Best finishing position

- Sort the results by total points in descending order
- Display the top 10 drivers based on total points

In [47]:
# Write your answer here

driver_agg = (results.groupby("driverId")
              .agg(avg_finish_pos = ("positionOrder", "mean"),
                   total_points = ("points", "sum"),
                   num_race_compl = ("driverId", len),
                   best_fin_pos = ("finish_category", "min"))
                   .sort_values(by="total_points", ascending=False))

driver_agg.head(10)

# ?? position order?

,avg_finish_pos,total_points,num_race_compl,best_fin_pos
driverId,,,,
1,4.787097,4396.5,310,Podium
20,7.093333,3098.0,300,Podium
4,8.494413,2061.0,358,Podium
830,6.533742,1983.5,163,Podium
8,8.491477,1873.0,352,Podium
822,7.601990,1778.0,201,Podium
3,8.252427,1594.5,206,Podium
30,6.879870,1566.0,308,Podium
817,9.883621,1307.0,232,Podium


### (e) Linear Regression
Create a linear regression model that predicts 'points' based on 'grid' (starting position) and 'laps' completed <br>
Use the following steps:

- Clean the data to remove any non-numeric values and missing values
- Create the regression formula using smf.ols 
- Display the summary of the regression model using model.summary()

What is the predicted points for a driver starting in position 3 and completing 55 laps?

Hint: Use ```.dropna()''' to remove missing values from the points, grid, and laps <br>
variables.

In [58]:
# Write your answer here

list1 = ["points", "grid", "laps"]
for i in list1:
    results[i].dropna()

model = smf.ols(formula = 'points ~ grid+laps', data = results)
outcome = model.fit()
outcome.summary()

b_list = outcome.params
print(b_list)

pred_points = b_list[0] + b_list[1]*(3) + b_list[2]*(55)
print(pred_points)

Intercept    2.584126
grid        -0.224796
laps         0.039279
dtype: float64
4.070062614810155
